# Clustering Townsville Suburbs

## Introduction  
Townsville is a major regional center in Queensland, Australia. It has a population of over 193,000. It is a major government, commercial and defence center. The Townsville local government area covers 3,736 square kilometres. The area considered in this analysis is about 180 sq klms. Population density in the urban area is just over 1060 persons per sq km.
This project will seek to cluster the urban suburbs of Townsville. The features used in the cluster analysis can broadly be considered as Lifestyle factors. They are: Socio-Economic Indexes; Access to Public Transport; Animal Complaints; Crime Data; Access to Council Community Facilities; Access to Community Venues.  
The insights gained from this project would be useful to people moving too, or moving within the Townsville District. It can give people an idea of commonalities and differences between suburbs.


## Data Sources:   
Data was sourced from several online sites, in Excel and Json file formats.  
Excel and Json files from the Australian Bureau of Statistics and the Townsville City Council Website.  
Json files retrieved in response to API requests to Qld State Government Crime Database and FourSquare database.  
Some data preparation was done in the Excel files beforehand.  
Geolocation data was obtained using the OpenStreetMap Nominatim API.  

In [ ]:
#Import all packages required

import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests

pd.options.display.float_format = '{:.2f}'.format

import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cmx

#%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import folium # map rendering library

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc
from sklearn.decomposition import PCA

In [ ]:
# Load suburb names. These were cut and paste from council website into Excel. This allowed for editing of required suburbs
# This was the most efficient method to get this list

df = pd.read_excel('townsville_suburbs.xlsx')
df['Suburb'] = df['Suburb'].str.strip() # strip leading and trailing spaces

In [ ]:
# get Townsville latitude and longitude

address = 'Townsville, Australia'

geolocator = Nominatim(user_agent="townsville_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Townsville are {}, {}.'.format(latitude, longitude))

In [ ]:
# Add suburb latitude and longitude

# function to return each suburbs location
def findlocation(sub):
    city = ', Townsville'
    address = sub+city
    geolocator = Nominatim(user_agent="townsville_explorer")
    location = geolocator.geocode(address)
    return location 

for ind in df.index:
    locate = findlocation(df.loc[ind][0])
    df.loc[[ind],'latitude'] = locate.latitude
    df.loc[[ind],'longitude'] = locate.longitude

In [ ]:
# Add SocioEconomic Data. Excel spreadsheet sourced from the Australian Bureau of Statistics. Prepared in Excel.
# Socioeconomic factors include:
# Index of Relative Socio-economic Disadvantage (IRSD)
# Index of Relative Socio-economic Advantage and Disadvantage (IRSAD)
# Index of Economic Resources (IER) 
# Index of Education and Occupation (IEO)
# A higher number is better.

se_df = pd.read_excel('state_suburb_socioeconomic_indexs_2016.xlsx', sheet_name='Sheet1',index_col=None,na_values=['NA'],usecols = "A:E")
se_df['Suburb'] = se_df['Suburb'].str.strip() # strip leading and trailing spaces from suburb names.
df_ec = pd.merge(df, se_df, on='Suburb', how='inner')

# convert object data to numeric for processing
df_ec['rsed'] = pd.to_numeric(df_ec['rsed'])
df_ec['rsead'] = pd.to_numeric(df_ec['rsead'])
df_ec['ier'] = pd.to_numeric(df_ec['ier'])
df_ec['ieo'] = pd.to_numeric(df_ec['ieo'])

In [ ]:
# Add bus shelters. Excel spreadsheet from City Council website.
# Counts the number of bus stops/shelters within a set radius of the suburb center.
# More bus stops indicate better access to public transport

pt_df = pd.read_excel('Bus_Shelters_Townsville.xlsx', sheet_name='Sheet1',index_col=None,na_values=['NA'],usecols = "A:G")

for ind in df_ec.index:
    count_stop = 0
    for ind2 in pt_df.index:
        coords_1 = (df_ec.iloc[ind,1],df_ec.iloc[ind,2])
        coords_2 = (pt_df.iloc[ind2,3],pt_df.iloc[ind2,4])
        #print(coords_1,coords_2)
        if (geopy.distance.distance(coords_1, coords_2).m) <= 500:
            count_stop = count_stop + 1
    df_ec.loc[[ind],'bus_stop'] = count_stop


In [ ]:
# Add Animal Complaints. Excel spreadsheet from City Council website.
# This tallies animal(dog) complaints delt with by the Council for the financial year 2019-2020
# Dataset is based on the 6 primary complaint categories of
# noise, wandering, enclosure, aggressive animal, attack and private impound.

ac_df = pd.read_excel('animal-complaints.xlsx', sheet_name='Sheet3',index_col=None,na_values=['NA'],usecols = "A:H")
ac_df['Suburb'] = ac_df['Suburb'].str.strip()
ac_df = ac_df.fillna(0)
ac_df.head()
df_ec_ac = pd.merge(df_ec, ac_df, on='Suburb', how='inner')

In [ ]:
# Add crime data. Uses API to access State crime database. Response is a json format file. Extracts data from Json file,
# then counts incident as either personal crime or property crime.

def get_crimes(suburb):
    property_crimes=['Arson', 'Other Property Damage', 'Robbery', 'Stock Related Offences', 'Trespassing and Vagrancy', 'Unlawful Entry', 'Unlawful Use of Motor Vehicle', 'Other Theft (excl. Unlawful Entry)']
    person_crimes=['Assault', 'Drug Offences', 'Fraud', 'Gaming Racing & Betting Offences', 'Good Order Offences', 'Handling Stolen Goods', 'Homicide (Murder)', 'Liquor (excl. Drunkenness)', 'Miscellaneous Offences', 'Other Homicide', 'Other Offences Against the Person', 'Prostitution Offences', 'Traffic and Related Offences', 'Weapons Act Offences']
    prop_cc = 0
    pers_cc = 0
    url = 'https://a5c7zwf7e5.execute-api.ap-southeast-2.amazonaws.com/dev/offences?locationType=SUBURB&startDate=07-01-2019&locationName={}&endDate=06-30-2020&format=JSON'.format(suburb)
    results = requests.get(url).json()
    crimes = []
    for ind in range(len(results)):
        crimes.append(results[ind]['Type'])
        
    for ind in range(len(crimes)):
        if any(crimes[ind] in x for x in property_crimes):
            prop_cc = prop_cc +1
        elif any(crimes[ind] in x for x in person_crimes):
            pers_cc = pers_cc + 1
    return(prop_cc,pers_cc)

for ind in range(len(df_ec_ac)):
    crime_counts = get_crimes(df_ec_ac.iloc[ind,0])
    df_ec_ac.loc[[ind],'prop_cc'] = crime_counts[0]
    df_ec_ac.loc[[ind],'pers_cc'] = crime_counts[1]

In [ ]:
# Add community facilities data. Counts council facilities (community halls, pools, library etc) within set distance from center of suburb

with open('TCC_Community_Facilities.json') as json_file:
       data = json.load(json_file)
data2 = data['features']

cf_df = pd.DataFrame(columns=['category', 'fac_lat', 'fac_long'])
for ind in range(len(data2)):
    row = [data['features'][ind]['properties']['category'], data['features'][ind]['geometry']['coordinates'][1], data['features'][ind]['geometry']['coordinates'][0]]
    cf_df.loc[len(cf_df)] = row

for ind in df_ec_ac.index:
    count_cf = 0
    for ind2 in cf_df.index:
        coords_1 = (df_ec_ac.iloc[ind,1],df_ec_ac.iloc[ind,2])
        coords_2 = (cf_df.iloc[ind2,1],cf_df.iloc[ind2,2])
        #print(coords_1,coords_2)
        if (geopy.distance.distance(coords_1, coords_2).m) <= 1000: #council facilities within 1000m
            count_cf = count_cf + 1
    df_ec_ac.loc[[ind],'council_fac'] = count_cf   

This next section uses FourSquare API to get venue details for each suburb. It counts the number of venues in different categories.

In [ ]:
CLIENT_ID = '************************' # your Foursquare ID
CLIENT_SECRET = '******************' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# Venues are summarized into categories of interest.
# Venues were within 1000m of the suburb center. A limit of 500 venues was retrieved per suburb. This was never reached.
# While the FourSquare database is useful, it is not well supported in remote, non-USA locations.

# Venue categories of interest
cafe = ['Café','Coffee Shop', 'Sandwich Place','Pizza Place','Juice Bar','Ice Cream Shop','Fried Chicken Joint','Burger Joint']
restaurant = ['Restaurant', 'Steakhouse']
supermarket = [ 'Supermarket', 'Market','Grocery Store']
pub = [ 'Pub','Liquor Store','Brewery']
entertainment = [ 'Theater', 'Multiplex']
recreation = [ 'Pool','Historic Site','Football Stadium','Bowling Alley','Basketball Stadium','Art Museum','Aquarium','Beach']

# process each suburb for venues
def process_suburb(suburb_result):
    suburb_venues = []
    for i in range(len(results_sub)):
        venue_found = results_sub[i]['venue']['categories'][0]['name']
        suburb_venues.append(venue_found)
    suburb_venues_new = [('Restaurant') if "Restaurant" in item else item for item in suburb_venues]
    cafe_count = sum(el in cafe for el in suburb_venues_new)
    restaurant_count = sum(el in restaurant for el in suburb_venues_new)
    supermarket_count = sum(el in supermarket for el in suburb_venues_new)
    pub_count = sum(el in pub for el in suburb_venues_new)
    entertainment_count = sum(el in entertainment for el in suburb_venues_new)
    recreation_count = sum(el in recreation for el in suburb_venues_new)
    
    return(cafe_count, restaurant_count, supermarket_count, pub_count, entertainment_count, recreation_count)

radius = 1000
LIMIT = 500

for ind in df_ec_ac.index:
    lat_test = df_ec_ac.iloc[ind,1]
    long_test = df_ec_ac.iloc[ind,2]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat_test, long_test, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    results_sub = results['response']['groups'][0]['items']
    venue_counts = process_suburb(results_sub)
    df_ec_ac.loc[[ind],'cafe_count'] = venue_counts[0]
    df_ec_ac.loc[[ind],'restaurant_count'] = venue_counts[1]
    df_ec_ac.loc[[ind],'supermarket_count'] = venue_counts[2]
    df_ec_ac.loc[[ind],'pub_count'] = venue_counts[3]    
    df_ec_ac.loc[[ind],'entertainment_count'] = venue_counts[4]
    df_ec_ac.loc[[ind],'recreation_count'] = venue_counts[5]    


In [ ]:
df_ec_ac.head()

In [ ]:
df_ec_ac.info()

## Exploratory Data Analysis

In [ ]:
column_Names = list(df_ec_ac.columns.values)

In [ ]:
df_ec_ac[column_Names[3:15]].describe()

In [ ]:
df_ec_ac[column_Names[15:24]].describe()

In [ ]:
print('Animal complaints are worse in:' , '\n\r', df_ec_ac.nlargest(3,'Animal Complaints Grand Total')['Suburb'])
ax = df_ec_ac.plot.bar(x='Suburb', y=['Animal Complaints Grand Total'], rot=90)

In [ ]:
print('Property Crime is worse in:' , '\n\r', df_ec_ac.nlargest(3,'prop_cc')['Suburb'])
print('Personal Crime is worse in:' , '\n\r', df_ec_ac.nlargest(3,'pers_cc')['Suburb'])
ax = df_ec_ac.plot.bar(x='Suburb', y=['prop_cc','pers_cc'], rot=90)

In [ ]:
print('The suburbs best serviced with Public Transport are:' , '\n\r', df_ec_ac.nlargest(3,'bus_stop')['Suburb'])
ax = df_ec_ac.plot.bar(x='Suburb', y=['bus_stop'], rot=90)

In [ ]:
df_ec_ac['Venue Total'] = df_ec_ac.iloc[:, -6:].sum(axis=1)
print('The suburbs best serviced with most Venues are:' , '\n\r', df_ec_ac.nlargest(3,'Venue Total')['Suburb'])
ax = df_ec_ac.plot.bar(x='Suburb', y=['Venue Total'], rot=90)
del df_ec_ac['Venue Total']

In [ ]:
corr = df_ec_ac.corr()
corr.style.background_gradient(cmap='coolwarm')

There is a strong correlation between the number of restaurants (type) venues and the number of cafe (type) venues.
There is a moderate correlation between the number of Personal Crime incidents and the number of Pub (type) venues.
There is a moderate correlation between the number of Property Crimes incidents and the number of Animal Noise Complaints. 

## Cluster Analysis

In [ ]:
#prepare data for KMeans

X = df_ec_ac.values[:,3:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)

In [ ]:
# function returns the WSS score (squared erros for all points)-used to identify elbow point; 
# the Inertia score (spread of points within a cluster)-used to identify elbow point;
# the Silhouette score (how far between clusters)-greater value is better clustering;
# for values of k between 1 to kmax

def calculate_Errors(points, kmax):
    init={}
    sse = []
    sil = []
    for k in range(1, kmax+1):
        kmeans = KMeans(n_clusters = k).fit(points)
        if k < 2:
            sil.append(0)
        else:
            labels = kmeans.labels_
            sil.append(silhouette_score(points, labels, metric = 'euclidean'))
        init[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
        centroids = kmeans.cluster_centers_
        pred_clusters = kmeans.predict(points)
        curr_sse = 0

        # calculate square of Euclidean distance of each point from its cluster center and add to current WSS
        for i in range(len(points)):
            curr_center = centroids[pred_clusters[i]]
            curr_sse += (points[i, 0] - curr_center[0]) ** 2 + (points[i, 1] - curr_center[1]) ** 2
        sse.append(curr_sse)

    return (sse,init,sil)

In [ ]:
# function to plot errors and Silhouette score

def plot_Errors(sse_errors2,sil_error2,init_errors):

    plt.figure(figsize=(12, 6))

    fig, ax1 = plt.subplots()

    ax2 = ax1.twinx()

    ax1.set_xlabel('K value')
    ax1.set_ylabel('SSE and Inertia', color='b')
    ax2.set_ylabel('Silhoutte index', color='brown')

    ax1.plot(range(1, kmax+1), sse_errors2, color='red', linestyle='dashed', marker='o',
             markerfacecolor='blue', markersize=10, label="sse errors: identify elbow point")

    ax1.plot(range(1, kmax+1), list(init_errors.values()), color='green', linestyle='dashed', marker='o',
             markerfacecolor='yellow', markersize=10, label="inertia measure: identify elbow point")

    ax2.plot(range(1, kmax+1), sil_error2, color='brown', linestyle='dashed', marker='o',
             markerfacecolor='violet', markersize=10, label="silhouette measure: greater is better")


    plt.title('Error Measures and K Values')
    plt.xticks(range(1,kmax+1))

    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc=1)

In [ ]:
#full dataset errors and Silhouette

kmax = 20
sserrors_all = calculate_Errors(cluster_dataset,kmax)

#extract metrics from returned tuple
raw_sse_errors = sserrors_all[0]
raw_init_errors = sserrors_all[1]
raw_sil_errors = sserrors_all[2]

#scale metric so they appear on same plot
raw_sse_errors2 = [i * 10 for i in raw_sse_errors]
raw_sil_error2 = [i * 1 for i in raw_sil_errors]

plot_Errors(raw_sse_errors2,raw_sil_error2,raw_init_errors)

In [ ]:
#PCA errors and Silhouette

pca = PCA(n_components=3)
pca_data = StandardScaler().fit_transform(X)
principalComponents = pca.fit_transform(pca_data)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])

kmax = 20
sserrors_all = calculate_Errors(principalComponents,kmax)

#extract metrics from returned tuple
pca_sse_errors = sserrors_all[0]
pca_init_errors = sserrors_all[1]
pca_sil_errors = sserrors_all[2]

#scale metric so they appear on same plot
pca_sse_errors2 = [i * 10 for i in pca_sse_errors]
pca_sil_error2 = [i * 1 for i in pca_sil_errors]

plot_Errors(pca_sse_errors2,pca_sil_error2,pca_init_errors)

In [ ]:
#plot dendrogram to identify number of clusters to use

D = df_ec_ac.values[:,3:]

plt.figure(figsize=(10, 7))
plt.title("Suburbs Dendograms")
dend = shc.dendrogram(shc.linkage(D, method='ward'))


## Selection of the best number of Clusters to use  
This is a very small dataset for KMeans modelling. I could not find any definitive answer regarding using KMeans on such a small dataset. The lack of a clearly identifiable best k value in the Error plots and the Silhouette Score most likely is due to the small data set. I did try reducing the number of features manually, by summing some features and removing some strongly correlated features, but this did not improve the result. A Principal Components Analysis was done to obtain a much smaller feature set. This gave a better case count to feature set ratio. The error plots and Silhouette Score for the PCA dataset did help to select a k value. Dendrogram plot helped justify the use of 3 clusters.

In [ ]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_


In [ ]:
%matplotlib notebook

In [ ]:
# Visualize KMeans result for PCA dataset using the Kmeans derived labels (clusters)

# Add Labels to PCA result dataframe
principalDf["Labels"] = labels

def scatter3d(x,y,z, cs, colorsMap='jet'):
    cm = plt.get_cmap(colorsMap)
    cNorm = matplotlib.colors.Normalize(vmin=min(cs), vmax=max(cs))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)
    fig = plt.figure()
    #ax = Axes3D(fig)
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, c=scalarMap.to_rgba(cs))
    scalarMap.set_array(cs)
    fig.colorbar(scalarMap)
    ax.set_xlabel('principal component 1', fontsize = 10)
    ax.set_ylabel('principal component 2', fontsize = 10)
    ax.set_zlabel('principal component 3', fontsize = 10)
    plt.show()
    
scatter3d(principalDf['principal component 1'],principalDf['principal component 2'],principalDf['principal component 3'],principalDf['Labels'])

Though poorly defined, it is possible to identify the 3 clusters.

In [ ]:
%matplotlib inline

In [ ]:
# Add cluster labels to main dataset

df_ec_ac["Labels"] = labels
df_ec_ac.head()

In [ ]:
# create map of Townsville using latitude and longitude values

f = folium.Figure(width=800, height=500)

map_townsville = folium.Map(location=[latitude, longitude], tiles="openstreetmap", zoom_start=11).add_to(f)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to map
markers_colors = []
for lat, lng, suburb, cluster in zip(df_ec_ac['latitude'], df_ec_ac['longitude'], df_ec_ac['Suburb'], df_ec_ac['Labels']):
    #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    label = '{}'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_townsville)  
    
map_townsville

In [ ]:
# print suburb clusters

for k in range(0, num_clusters):
    print('Suburbs in Group ',k,'\n\r',df_ec_ac.loc[df_ec_ac['Labels'] == k].Suburb)

## Descriptive Statistics for the Different Suburbs ( the Label is the Cluster Number)

In [ ]:
df_ec_ac[["rsed",'rsead','ier','ieo','Labels']].groupby("Labels").agg({'rsed': ['min', 'max', 'median','mean'],
                                                                 'rsead': ['min', 'max', 'median', 'mean'],
                                                                 'ier': ['min', 'max', 'median', 'mean'],
                                                                 'ieo': ['min', 'max', 'median', 'mean']})

In [ ]:
df_ec_ac[["prop_cc", "pers_cc",'Animal Complaints Grand Total','Labels']].groupby("Labels").agg({'prop_cc': ['min', 'max', 'median','mean'],
                                                                 'pers_cc': ['min', 'max', 'median', 'mean'],
                                                                 'Animal Complaints Grand Total': ['min', 'max', 'median', 'mean'],})

In [ ]:
df_ec_ac[["cafe_count", "restaurant_count",'Labels']].groupby("Labels").agg({'cafe_count': ['min', 'max', 'median','mean'],
                                                                 'restaurant_count': ['min', 'max', 'median', 'mean']})

In [ ]:
df_ec_ac[["supermarket_count",'pub_count','entertainment_count','recreation_count','Labels']].groupby("Labels").agg({'supermarket_count': ['min', 'max', 'median','mean'],
                                                                 'pub_count': ['min', 'max', 'median', 'mean'],
                                                                 'entertainment_count': ['min', 'max', 'median', 'mean'],
                                                                 'recreation_count': ['min', 'max', 'median', 'mean']})

## Boxplots for each feature, for each cluster

In [ ]:
for ind in range(len(column_Names)):
    if (ind > 2):
        sns.boxplot( x=df_ec_ac["Labels"], y=df_ec_ac[column_Names[ind]], width=0.3);
        plt.show()

In [ ]:
%matplotlib inline